# 3DXRD segmentation notebook for Frelon-ish detectors  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will help you to extract the locations of diffraction peaks on your detector images.  
It will also merge together your 2D spots (on a stack of detector images with different omega angles).  
We merge across omega because we often see the same spot twice on multiple detector images.  
The results are saved to the PROCESSED_DATA folder of the experiment, inside the sample and dataset folders that you select within this notebook

__NOTE: These notebooks are under active development
They require the latest version of ImageD11 from Git to run.__  
If you don't have this set up yet, you can run the below cell.  
It will automatically download and install ImageD11 to your home directory

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# Experts : update these files for your detector if you need to

# give dx/dy as tuple instead of spline
# Since 2024: there is no good spline for a detector at ID11. You probably want to use an e2dx, e2dy file
# You can provide this as a simple string:
# splinefile = '/path/to/spline.spline'
# or as a tuple of strings for e2dx/e2dy files
splinefile = ('/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_dx.edf','/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_dy.edf')
bgfile = None
maskfile = '/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/mask.edf'
darkfile = "/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/dark_20240416.edf"
flatfile = "/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/F36_Nov2023.edf"

detector = "frelon3"  # fixme - guess this from masterfile + scan
omegamotor = "diffrz"
dtymotor = "diffty"

# Default segmentation options
options = {
    "bgfile":bgfile,
    "maskfile":maskfile,
    "darkfile":darkfile,
    "flatfile":flatfile,
    "threshold":70,
    "smoothsigma":1.0,
    "bgc":0.9,
    "minpx":3,
    "m_offset_thresh":100,
    "m_ratio_thresh":150,
}
normalise_intensities_to_monitor = True
monitor_name = 'fpico4'

# EXPERTS: These can be provided as papermill parameters. Users, leave these as None for now...
dataroot = None
analysisroot = None
sample = None
dataset = None
scans = ["1.1",]

In [ ]:
import fabio
import numpy as np
import matplotlib.pyplot as plt

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties
from ImageD11.nbGui import nb_utils as utils
from ImageD11.nbGui import segmenter_gui
from ImageD11.frelon_peaksearch import worker, segment_dataset, guess_bg

%matplotlib widget

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
if dataroot is None:
    dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
if sample is None:
    sample = 'sample'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
if dataset is None:
    dataset = "dataset"

In [ ]:
# create ImageD11 dataset object

ds = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector,
                                        omegamotor=omegamotor,
                                        dtymotor=dtymotor)
ds.import_all(scans=scans)
if isinstance(splinefile, (tuple, list)) and len(splinefile) == 1:
    # we have ("splinefile", )
    ds.splinefile = splinefile[0]  # take the splinefile out of the tuple
elif isinstance(splinefile, (tuple, list)):
    # we have (e2dx, e2dy)
    ds.e2dxfile, ds.e2dyfile = splinefile
else:
    # we have "splinefile"
    ds.splinefile = splinefile
ds.maskfile = maskfile
ds.bgfile = bgfile
ds.darkfile = darkfile
ds.flatfile = flatfile
ds.save()

In [ ]:
# normally not needed:

# bg = guess_bg( ds )
# plt.imshow(bg)
# fabio.edfimage.edfimage(bg).save('bg.edf')
# plt.colorbar()
# ds.bgfile = 'bg.edf'

In [ ]:
ui = segmenter_gui.FrelonSegmenterGui(ds, worker, segment_dataset, **options)

In [ ]:
options = ui.getopts()
print(options)

# Intensity normalisation
You can optionally normalise your observed intensities to a monitor column, such as a pico, if you had one in the beam path.

In [ ]:
if normalise_intensities_to_monitor:
    monitor_per_frame = ds.get_monitor(monitor_name)

    fig, ax = plt.subplots(layout='constrained')
    ax.plot(ds.omega[0], monitor_per_frame[0])
    ax.set(xlabel=r'$\omega~(\degree)$', ylabel='Intensity', title='Monitor column')
    plt.show()

To normalise to a monitor signal, we need to choose a "reference" monitor value that we scale to.
A good choice may be `np.mean(monitor_per_frame)`

We then compute `scale_factor_per_frame = np.mean(monitor_per_frame) / monitor_per_frame`  
We then multiply the observed intensities by `scale_factor_per_frame`

You can choose which function to use to generate the "reference" monitor value, for example:

`segment_dataset(..., monitor_name='fpico6', monitor_ref_func=np.mean)`

In [ ]:
# ensure that we don't normalise if you choose not to

if not normalise_intensities_to_monitor:
    monitor_name = None
    # ensure no monitor currently set
    ds.monitor = None
    ds.monitor_ref = None
    ds.reset_peaks_cache()

# Segment

In [ ]:
cf_2d, cf_3d = segment_dataset(ds, options, monitor_name=monitor_name)

In [ ]:
# display some peaks
f,a=plt.subplots(1,2,figsize=(12,6), layout='constrained')
a[0].plot(cf_3d.f_raw,cf_3d.s_raw,'.',ms=1)
a[0].set(xlabel='fast index', ylabel='slow index',aspect='equal', title='peaks on detector')
a[1].plot(cf_3d.omega,cf_3d.sum_intensity,'.',ms=1)
a[1].set(xlabel=r'$\omega~(\degree)$',ylabel='sum intensity',yscale='log',title='peaks vs omega')
plt.show()

In [ ]:
ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile)

In [ ]:
ds.save()